In [9]:
import json
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity


from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power

from itertools import product


pd.set_option('display.max_colwidth', None)

In [10]:
import pandas as pd
btc_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\btc.csv', index_col=None)
eth_df= pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\eth.csv',  index_col=None)
ada_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\ada.csv',  index_col=None)
sol_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\sol.csv',  index_col=None)

In [11]:
btc_with_sections_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\btc_with_sections.csv', index_col=None)

In [12]:
import nltk
btc_with_sections_df["sents"] =  btc_with_sections_df.apply(lambda x : nltk.tokenize.sent_tokenize(x["text"]), axis=1)

In [31]:
btc_with_sections_df.head(2)

,text,section,sents
0,"A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution. Digital signatures provide part of the solution, but the main benefits are lost if a trusted third party is still required to prevent double-spending. We propose a solution to the double-spending problem using a peer-to-peer network. The network timestamps transactions by hashing them into an ongoing chain of hash-based proof-of-work, forming a record that cannot be changed without redoing the proof-of-work. The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power. As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers. The network itself requires minimal structure. Messages are broadcast on a best effort basis, and nodes can leave and rejoin the network at will, accepting the longest proof-of-work chain as proof of what happened while they were gone.",Abstract,"[A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution., Digital signatures provide part of the solution, but the main benefits are lost if a trusted third party is still required to prevent double-spending., We propose a solution to the double-spending problem using a peer-to-peer network., The network timestamps transactions by hashing them into an ongoing chain of hash-based proof-of-work, forming a record that cannot be changed without redoing the proof-of-work., The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power., As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers., The network itself requires minimal structure., Messages are broadcast on a best effort basis, and nodes can leave and rejoin the network at will, accepting the longest proof-of-work chain as proof of what happened while they were gone.]"
1,"Commerce on the Internet has come to rely almost exclusively on financial institutions serving as trusted third parties to process electronic payments. While the system works well enough for most transactions, it still suffers from the inherent weaknesses of the trust based model. Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes. The cost of mediation increases transaction costs, limiting the minimum practical transaction size and cutting off the possibility for small casual transactions, and there is a broader cost in the loss of ability to make non-reversible payments for nonreversible services. With the possibility of reversal, the need for trust spreads. Merchants must be wary of their customers, hassling them for more information than they would otherwise need. A certain percentage of fraud is accepted as unavoidable. These costs and payment uncertainties can be avoided in person by using physical currency, but no mechanism exists to make payments over a communications channel without a trusted party. What is needed is an electronic payment system based on cryptographic proof instead of trust, allowing any two willing parties to transact directly with each other without the need for a trusted third party. Transactions that are computationally impractical to reverse would protect sellers from fraud, and routine escrow mechanisms could easily be implemented to protect buyers. In this paper, we propose a solution to the double-spending problem using a peer-to-peer distributed timestamp server to generate computational proof of the chronological order of transactions. The system is secure as long as honest 

In [7]:
btc_with_sections_df.columns

Index(['text', 'section', 'sents'], dtype='object')

In [13]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [14]:
def gen_match_matrix(model, sents, min_match_score=0):
    sents1 = [x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0
    return sim_matrix

In [84]:
def filter_ranked_list(ranked_sents, model, min_match=0.8, N=3):
    ranked_sents = [x[0] for x in ranked_sents]
    filtered_sents = []
    for i, s in enumerate(ranked_sents):
        if len(filtered_sents) >= N or len(filtered_sents) >= len(ranked_sents):
            break
        if len(filtered_sents) == 0:
            filtered_sents.append(s)
            # print('FFFF ' , s)
        else:
            matching_scores = gen_match_matrix(model, [s] + filtered_sents)
            max_sim = np.max(matching_scores[0][1:])
            # print(matching_scores)
            if max_sim < min_match:
                # print('SSSS ' , s)
                filtered_sents.append(s)
        
        # print(min_match)
    
    return filtered_sents

In [25]:
def apply_page_rank(sentences, p=0.85, min_match_score=0.5, min_len=5, max_len=35):
    cand_sents = [x for x in sentences if len(x.split()) < max_len and len(x.split()) > min_len]
    if len(cand_sents) == 0:
        print('empty')
        return []
    # print(cands)
    # print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cand_sents, min_match_score=min_match_score)
    pr = pagerank(cands_matching_mat, p=p)  # it looks like modifying the initial probability doesn't help
    # pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cand_sents, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])

In [85]:
ranked_sents_per_section = []

sections = []
ranked_sents_per_section = []
ranked_kps_per_section = []
for i, row in btc_with_sections_df.iterrows():
    ranked_sents = apply_page_rank(row['sents'], p=0.2, min_match_score=0.5)
    ranked_sents_per_section.append(ranked_sents)
    ranked_kps = filter_ranked_list(ranked_sents, model, min_match=0.8, N=2)
    ranked_kps_per_section.append(ranked_kps)
    sections.append(row["section"])
    
results_df = pd.DataFrame({"sections": sections, "ranked_sents": ranked_sents_per_section, "ranked_kps": ranked_kps_per_section})


C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages

In [86]:
results_df[["sections", "ranked_kps"]].head(10)

,sections,ranked_kps
0,Abstract,"[Messages are broadcast on a best effort basis, and nodes can leave and rejoin the network at will, accepting the longest proof-of-work chain as proof of what happened while they were gone., A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution.]"
1,Introduction,"[Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes., Commerce on the Internet has come to rely almost exclusively on financial institutions serving as trusted third parties to process electronic payments.]"
2,Transactions,"[After each transaction, the coin must be returned to the mint to issue a new coin, and only coins issued directly from the mint are trusted not to be double-spent., The problem with this solution is that the fate of the entire money system depends on the company running the mint, with every transaction having to go through them, just like a bank.]"
3,Timestamp Server,"[The solution we propose begins with a timestamp server., A timestamp server works by taking a hash of a block of items to be timestamped and widely publishing the hash, such as in a newspaper or Usenet post [2-5].]"
4,Proof-of-Work,"[To modify a past block, an attacker would have to redo the proof-of-work of the block and all blocks after it and then catch up with and surpass the work of the honest nodes., To compensate for increasing hardware speed and varying interest in running nodes over time, the proof-of-work difficulty is determined by a moving average targeting an average number of blocks per hour.]"
5,Network,"[2) Each node collects new transactions into a block., 4) When a node finds a proof-of-work, it broadcasts the block to all nodes.]"
6,Incentive,"[The incentive can also be funded with transaction fees., By convention, the first transaction in a block is a special transaction that starts a new coin owned\nby the creator of the block.]"
7,Reclaiming Disk Space,"[Once the latest transaction in a coin is buried under enough blocks, the spent transactions before it can be discarded to save disk space., To facilitate this without breaking the block's hash, transactions are hashed in a Merkle Tree [7][2][5], with only the root included in the block's hash.]"
8,Simplified Payment Verification,"[As such, the verification is reliable as long as honest nodes control the network, but is more vulnerable if the network is overpowered by an attacker., It is possible to verify payments without running a full network node.]"
9,Combining and Splitting Value,"[Although it would be possible to handle coins individually, it would be unwieldy to make a separate transaction for every cent in a transfer., To allow value to be split and combined, transactions contain multiple inputs and outputs.]"


In [ ]:
ranked_sents_per_section

In [35]:
all_text = []
for i, row in btc_with_sections_df.iterrows():
    all_text.extend(row["sents"])

In [51]:
# with all text 
ranked_sents = apply_page_rank(all_text, p=0.2, min_match_score=0.5)
ranked_kps = filter_ranked_list(ranked_sents, model, min_match=0.8, N=20)

C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [52]:
# all text ranked
ranked_kps

['As later blocks are chained after it, the work to change the block would include redoing all the blocks after it.',
 '2) Each node collects new transactions into a block.',
 '5) Nodes accept the block only if all transactions in it are valid and not already spent.',
 "As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers.",
 'Once the CPU effort has been expended to make it satisfy the proof-of-work, the block cannot be changed without redoing the work.',
 'In this paper, we propose a solution to the double-spending problem using a peer-to-peer distributed timestamp server to generate computational proof of the chronological order of transactions.',
 'A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution.',
 'To modify a past block, an attacker would have to redo 

In [54]:
# clusered
clustered_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\agglomerative_clustering_btc.csv")

In [76]:
from ast import literal_eval
clustered_df["sentences"] = clustered_df["sentences"].apply(lambda x: literal_eval(x))

In [77]:
clustered_df["sentences"].head(1)

0    [A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution., Commerce on the Internet has come to rely almost exclusively on financial institutions serving as trusted third parties to process electronic payments., Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes., The cost of mediation increases transaction costs, limiting the minimum practical transaction size and cutting off the possibility for small casual transactions, and there is a broader cost in the loss of ability to make non-reversible payments for nonreversible services., These costs and payment uncertainties can be avoided in person by using physical currency, but no mechanism exists to make payments over a communications channel without a trusted party., Transactions that are computationally impractical to reverse would protect seller

In [64]:
cand_sents

[]

In [87]:
cluster_ids = []
ranked_sents_per_cluster = []
ranked_kps_per_cluster = []
for i, row in clustered_df.iterrows():
    ranked_sents = apply_page_rank(row['sentences'], p=0.2, min_match_score=0.5)
    ranked_sents_per_cluster.append(ranked_sents)
    ranked_kps = filter_ranked_list(ranked_sents, model, min_match=0.8, N=2)
    ranked_kps_per_cluster.append(ranked_kps)
    cluster_ids.append(row["cluster_id"])
    
results_clustered_df = pd.DataFrame({"clusters": cluster_ids, "ranked_sents": ranked_sents_per_cluster, "ranked_kps": ranked_kps_per_cluster})

C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\AppData\Roaming\Python\Python39\site-packages

In [88]:
results_clustered_df[["clusters", "ranked_kps"]].head(10)

,clusters,ranked_kps
0,6,"[Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes., After each transaction, the coin must be returned to the mint to issue a new coin, and only coins issued directly from the mint are trusted not to be double-spent.]"
1,1,"[We started with the usual framework of coins made from digital signatures, which provides strong control of ownership, but is incomplete without a way to prevent double-spending., The majority decision is represented by the longest chain, which has the greatest proof-of-work effort invested in it.]"
2,8,"[In this paper, we propose a solution to the double-spending problem using a peer-to-peer distributed timestamp server to generate computational proof of the chronological order of transactions., A timestamp server works by taking a hash of a block of items to be timestamped and widely publishing the hash, such as in a newspaper or Usenet post [2-5].]"
3,4,"[As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers., Nodes can leave and rejoin the network at will, accepting the proof-of-work chain as proof of what happened while they were gone.]"
4,0,"[The network itself requires minimal structure., With the possibility of reversal, the need for trust spreads.]"
5,5,"[While the system works well enough for most transactions, it still suffers from the inherent weaknesses of the trust based model., The public can see that someone is sending an amount to someone else, but without information linking the transaction to anyone.]"
6,9,"[Each owner transfers the coin to the next by digitally signing a hash of the previous transaction and the public key of the next owner and adding these to the end of the coin., By convention, the first transaction in a block is a special transaction that starts a new coin owned\nby the creator of the block.]"
7,2,"[2) Each node collects new transactions into a block., As later blocks are chained after it, the work to change the block would include redoing all the blocks after it.]"
8,3,"[For our timestamp network, we implement the proof-of-work by incrementing a nonce in the block until a value is found that gives the block's hash the required zero bits., Once the CPU effort has been expended to make it satisfy the proof-of-work, the block cannot be changed without redoing the work.]"
9,7,"[We will show later that the probability of a slower attacker catching up diminishes exponentially as subsequent blocks are added., The race between the honest chain and an attacker chain can be characterized as a Binomial Random Walk.]"
